In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [17]:
review = list()

with open('yelp_academic_dataset_review.json', encoding='utf-8') as file:
    for line in file:
#         print(line)
        review.append(json.loads(line))

reviews = pd.DataFrame(review)

In [18]:
len(reviews)

6990280

In [21]:
reviews.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [23]:
review_final = reviews[['review_id', 'business_id', 'stars', 'text']]

In [24]:
review_final = review_final.rename(columns={'stars':'review_stars', 'text':'review_text'})

In [30]:
# binary = False
# tri = True

# Function to assign sentiment
def get_sentiment(stars, bi=True, tri=False):
    assert stars >= 1 and stars <= 5 and type(stars) == int
    if bi:
        if stars <=2:
            return 0
        else:
            return 1
    elif tri:
        if stars <= 2:
            return -1
        elif stars == 3:
            return 0
        else:
            return 1
    else:
        return int(n)


In [26]:
review_final.head()

,review_id,business_id,review_stars,review_text
0,KU_O5udG6zpxOg-VcAEodg,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,"If you decide to eat here, just be aware it is..."
1,BiTunyQ73aT9WBnpR9DZGw,7ATYjTIgM3jUlt4UM3IypQ,5.0,I've taken a lot of spin classes over the year...
2,saUsX_uimxRlCVr67Z4Jig,YjUWPpI6HXG530lwP-fb2A,3.0,Family diner. Had the buffet. Eclectic assortm...
3,AqPFMleE6RsU23_auESxiA,kxX2SOes4o-D3ZQBkiMRfA,5.0,"Wow! Yummy, different, delicious. Our favo..."
4,Sx8TMOWLNuJBWer-0pcmoA,e4Vwtrqf-wpJfwesgvdgxQ,4.0,Cute interior and owner (?) gave us tour of up...


In [27]:
dataset = review_final.loc[:,['review_stars', 'review_text']]
dataset.head()

,review_stars,review_text
0,3.0,"If you decide to eat here, just be aware it is..."
1,5.0,I've taken a lot of spin classes over the year...
2,3.0,Family diner. Had the buffet. Eclectic assortm...
3,5.0,"Wow! Yummy, different, delicious. Our favo..."
4,4.0,Cute interior and owner (?) gave us tour of up...


In [28]:
dataset = dataset.astype({'review_stars': 'int32'})

In [31]:
dataset['sentiment'] = dataset['review_stars'].apply(lambda x : get_sentiment(x, bi=True))
dataset.head()

,review_stars,review_text,sentiment
0,3,"If you decide to eat here, just be aware it is...",1
1,5,I've taken a lot of spin classes over the year...,1
2,3,Family diner. Had the buffet. Eclectic assortm...,1
3,5,"Wow! Yummy, different, delicious. Our favo...",1
4,4,Cute interior and owner (?) gave us tour of up...,1


In [32]:
dataset['sentiment'].value_counts()

1    5376479
0    1613801
Name: sentiment, dtype: int64

In [33]:
dataset = dataset.reset_index(drop=True)

In [34]:
# Remove stars column
dataset = dataset.drop('review_stars', axis=1)
dataset = dataset.rename(columns={'review_text':'review'})
dataset.head()

,review,sentiment
0,"If you decide to eat here, just be aware it is...",1
1,I've taken a lot of spin classes over the year...,1
2,Family diner. Had the buffet. Eclectic assortm...,1
3,"Wow! Yummy, different, delicious. Our favo...",1
4,Cute interior and owner (?) gave us tour of up...,1


In [35]:
dataset.describe()

,sentiment
count,6.990280e+06
mean,7.691364e-01
std,4.213853e-01
min,0.000000e+00
25%,1.000000e+00
50%,1.000000e+00
75%,1.000000e+00
max,1.000000e+00


In [40]:
# slice dataset to take top 10000 of each sentiment
dataset_positive = dataset[dataset['sentiment'] == 1].head(5000)
dataset_negative = dataset[dataset['sentiment'] == 0].head(5000)
dataset_reduced = pd.concat([dataset_positive, dataset_negative])

In [41]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataset, train_size=0.75)

train.describe()
test.describe()

print("Exporting training & testing data")
train.to_json('yelp_train_reduced.json', orient='records')
test.to_json('yelp_test_reduced.json', orient='records')

Exporting training & testing data


OverflowError: Could not reserve memory block

In [ ]:
# Import pretrained Embeddings
import os
import urllib.request

print("Importing embeddings")
urllib.request.urlretrieve('http://nlp.stanford.edu/data/glove.6B.zip', 'glove.6B.zip')

print("unzipping embeddings")
# !unzip glove.6B.zip

urllib.request.urlretrieve('http://nlp.stanford.edu/data/glove.twitter.27B.zip', 'glove.twitter.27B.zip')
# !unzip glove.6B.zip